<a href="https://colab.research.google.com/github/rkdwogns/TIS/blob/main/%EA%B5%90%EC%B0%A8_%EA%B2%80%EC%A6%9D%2C_%EA%B7%B8%EB%A6%AC%EB%93%9C_%EC%84%9C%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [2]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)

In [4]:
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

In [5]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [6]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [7]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.01137447, 0.01068664, 0.01105571, 0.0109868 , 0.0097177 ]), 'score_time': array([0.00131941, 0.00167346, 0.00141644, 0.00156093, 0.00135732]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [8]:
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


In [9]:
# 앞서 수행한 교차검증은 다음 코드와 동일

from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold()) # StratifiedKFold()가 기본 설정
print(np.mean(scores['test_score']))

0.855300214703487


In [10]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [11]:
from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [12]:
# GridSearchCV 클래스에 탐색 대상 모델과 params 변수를 전달하여 그리드 서치 객체를 만듦

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
# n_jobs=-1은 시스템에 있는 모든 코어를 사용함을 의미

In [13]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [14]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [15]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [16]:
# 5번의 교차 검증으로 얻은 점수를 출력

print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [17]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [30]:
# 연습문제
# 다음 넘파이 배열의 가장 작은 값을 찾아 해당 값의 제곱을 구하시오.
# 단, np.argmin() 함수 사용


import numpy as np
np.random.seed(30)

arr = np.random.randn(1000)
# print(arr)
x = np.argmin(arr)
arr[24]**2

8.429543401048143

In [34]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

In [35]:
# 싸이파이(scipy) 라이브러리에서 2개의 확률분포를 임포트
# uniform: 주어진 범위에서 실수값을 고르게 추출(샘플링)
# randint: 주어진 범위에서 정수값을 고르게 추출(샘플링)

from scipy.stats import uniform, randint

In [36]:
# 0~10 범위의 randint 객체를 만들고 10개 숫자를 샘플링

rgen = randint(0, 10)
rgen.rvs(10) # 랜덤 표본 생성(rvs: random variable sampling)

array([1, 4, 3, 1, 3, 7, 0, 4, 5, 4])

In [37]:
# 0~1 범위에서 10개 실수를 추출

ugen = uniform(0, 1)
ugen.rvs(10)

array([0.91252146, 0.0097403 , 0.33418632, 0.28049732, 0.75922228,
       0.08001799, 0.97932686, 0.28804327, 0.12681494, 0.95406116])

In [38]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }